In [46]:
import numpy as np
import pandas as pd
import matplotlib.pyplot as plt
import seaborn as sns
from math import exp
%matplotlib inline

pd.set_option('display.max_rows', 500)
pd.set_option('display.max_columns', 300)
pd.set_option('display.width', 1000)
pd.set_option('display.float_format', lambda x: '%.3f' % x)

In [2]:
housing = pd.read_csv('full_ohe_no_outlier.csv', index_col=0)
housing.shape
housing.sample(5)

(1444, 284)

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_No_Alley,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CemntBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_No_Bsmt,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_No_Bsmt,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_No_G,PavedDrive_P,PavedDrive_Y,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_No_Fence,MiscFeature_No_MF,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_No_Bsmt,BsmtQual_TA,BsmtCond_Gd,BsmtCond_No_Bsmt,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_No_Bsmt,HeatingQC_Fa,HeatingQ

Outliers have been removed in jj_lr and df is fully OHE. Probably use min_max scaler (retest this at the end). Now we optimize for lasso/ridge alphas. See which features can be removed.

In [3]:
from sklearn.linear_model import Lasso, Ridge
from sklearn.preprocessing import MinMaxScaler
from sklearn.metrics import mean_squared_error
from sklearn.model_selection import KFold

continuous = ['LotFrontage','LotArea','MasVnrArea','BsmtFinSF1','BsmtFinSF2','BsmtUnfSF','1stFlrSF',
                 '2ndFlrSF','LowQualFinSF','GrLivArea','GarageArea','WoodDeckSF','OpenPorchSF','EnclosedPorch',
                  '3SsnPorch','ScreenPorch','PoolArea','MiscVal','YearBuilt','YearRemodAdd','GarageYrBlt']

min_max = MinMaxScaler()


In [4]:
housing[continuous] = min_max.fit_transform(housing[continuous])

In [5]:
random_state = 24
def score_log_lasso(alphas):
    score_log = []
    
    k6 = KFold(n_splits=6, shuffle=True, random_state=random_state)

    train = []
    test = []
    for train_idx, val_idx in k6.split(housing):
        train.append(train_idx)
        test.append(val_idx)
    
    for alpha in alphas:
        lasso = Lasso(random_state=random_state, alpha=alpha, max_iter=10000)
        for i in range(6):
            single_log = []
           
            X_train = housing.iloc[train[i],:-1]
            X_test = housing.iloc[test[i],:-1]
            y_train = housing.iloc[train[i],-1]
            y_test = housing.iloc[test[i],-1]
            
            lasso.fit(X_train, y_train)
            
            single_log.append(random_state)
            single_log.append(alpha)
            single_log.append(i)
            single_log.append(lasso.score(X_train, y_train))
            single_log.append(lasso.score(X_test, y_test))
            
            single_log.append(mean_squared_error(y_test,lasso.predict(X_test))**0.5)
            score_log.append(single_log)
    
    score_df = pd.DataFrame(score_log, columns=['RandomState','Alpha','Fold','Train','Test','MSE'])
    
    return score_df
            
            

In [6]:
random_state = 24
def score_log_ridge(alphas):
    score_log = []
    
    k6 = KFold(n_splits=6, shuffle=True, random_state=random_state)

    train = []
    test = []
    for train_idx, val_idx in k6.split(housing):
        train.append(train_idx)
        test.append(val_idx)
    
    for alpha in alphas:
        ridge = Ridge(random_state=random_state, alpha=alpha, max_iter=10000)
        for i in range(6):
            single_log = []
#             print('train {} elements'.format(len(train[i])))
#             print('test {} elements'.format(len(test[i])))

            X_train = housing.iloc[train[i],:-1]
            X_test = housing.iloc[test[i],:-1]
            y_train = housing.iloc[train[i],-1]
            y_test = housing.iloc[test[i],-1]
            
            ridge.fit(X_train, y_train)
            single_log.append(random_state)
            single_log.append(alpha)
            single_log.append(i)
            single_log.append(ridge.score(X_train, y_train))
            single_log.append(ridge.score(X_test, y_test))
            
            single_log.append(mean_squared_error(y_test, ridge.predict(X_test))**0.5)
            
            score_log.append(single_log)
    
    score_df = pd.DataFrame(score_log, columns=['RandomState','Alpha','Fold','Train','Test','MSE'])
    
    return score_df
            
            

In [7]:
lasso_scores = score_log_lasso(np.logspace(0,3,20))
lasso_scores

,RandomState,Alpha,Fold,Train,Test,MSE
0,24,1.000,0,0.943,0.910,24249.562
1,24,1.000,1,0.945,0.907,19119.616
2,24,1.000,2,0.944,0.904,23200.944
3,24,1.000,3,0.951,0.860,25414.491
4,24,1.000,4,0.944,0.908,26073.239
5,24,1.000,5,0.946,0.908,25064.247
6,24,1.438,0,0.943,0.910,24195.228
7,24,1.438,1,0.945,0.908,19028.648
8,24,1.438,2,0.944,0.905,23044.864
9,24,1.438,3,0.951,0.860,25369.761


In [8]:
lasso_scores = score_log_lasso(np.linspace(6,80,20))

In [9]:
lasso_scores.groupby('Alpha').mean()[['Test','MSE']]

,Test,MSE
Alpha,,
6.000,0.905,23172.632
9.895,0.907,22927.188
13.789,0.908,22754.409
17.684,0.909,22661.743
21.579,0.910,22606.885
25.474,0.910,22565.678
29.368,0.910,22540.927
33.263,0.910,22527.022
37.158,0.910,22522.910


optimal lasso alpha: 36

In [10]:
ridge_score = score_log_ridge(np.linspace(1,3,20))

In [11]:
ridge_score.groupby('Alpha').mean()[['Test','MSE']]

,Test,MSE
Alpha,,
1.000,0.907,22936.410
1.105,0.907,22918.039
1.211,0.907,22904.532
1.316,0.907,22895.064
1.421,0.908,22888.985
1.526,0.908,22885.779
1.632,0.908,22885.026
1.737,0.908,22886.380
1.842,0.908,22889.558


ridge optimal alpha = 1.6

In [12]:
from sklearn.model_selection import train_test_split
random_state =24
min_max =  MinMaxScaler()
housing[continuous] = min_max.fit_transform(housing[continuous])

X_train, X_test, y_train, y_test = train_test_split(housing.iloc[:,:-1], housing.iloc[:,-1],
                                                    test_size=0.25, random_state=random_state)

lasso = Lasso(alpha=60, random_state=random_state)
lasso.fit(X_train, y_train)
ridge = Ridge(alpha=1.6, random_state=random_state)
ridge.fit(X_train, y_train)

print('Lasso Test Score: {}'.format(lasso.score(X_test,y_test)))
print('Lasso MSE: {}'.format(mean_squared_error(y_test,lasso.predict(X_test))**0.5))
print('Ridge Test Score: {}'.format(ridge.score(X_test,y_test)))
print('Ridge MSE: {}'.format(mean_squared_error(y_test,ridge.predict(X_test))**0.5))

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=24, selection='cyclic',
      tol=0.0001, warm_start=False)

Ridge(alpha=1.6, copy_X=True, fit_intercept=True, max_iter=None,
      normalize=False, random_state=24, solver='auto', tol=0.001)

Lasso Test Score: 0.9159991557099485
Lasso MSE: 22544.9615746681
Ridge Test Score: 0.9208515676812187
Ridge MSE: 21884.107148480733


wrote a for loop to see what features is to remove around area of 33-64

In [13]:
from sklearn.model_selection import train_test_split
random_state =24
min_max =  MinMaxScaler()
housing[continuous] = min_max.fit_transform(housing[continuous])

X_train, X_test, y_train, y_test = train_test_split(housing.iloc[:,:-1], housing.iloc[:,-1],
                                                    test_size=0.25, random_state=random_state)
features_to_drop = {}

for i in range(33,64):
    lasso = Lasso(alpha=i, random_state=random_state)
    lasso.fit(X_train, y_train)
    drop_idx = []
    print('Alpha: {}'.format(i))
    print('Lasso Test Score: {}'.format(lasso.score(X_test,y_test)))
    print('Lasso MSE: {}'.format(mean_squared_error(y_test,lasso.predict(X_test))**0.5))
    for k in range(len(lasso.coef_)):
        if (lasso.coef_[k] <0.01) & (lasso.coef_[k] > -0.01):
            drop_idx.append(k)
    features_to_drop[i]=drop_idx

features_to_drop

Lasso(alpha=33, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=24, selection='cyclic',
      tol=0.0001, warm_start=False)

Alpha: 33
Lasso Test Score: 0.9171313377679883
Lasso MSE: 22392.513145927107


Lasso(alpha=34, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=24, selection='cyclic',
      tol=0.0001, warm_start=False)

Alpha: 34
Lasso Test Score: 0.9170627264438788
Lasso MSE: 22401.78119765403


C:\Users\Jimmy Jing\Anaconda3\lib\site-packages\sklearn\linear_model\coordinate_descent.py:475: ConvergenceWarning: Objective did not converge. You might want to increase the number of iterations. Duality gap: 1195739484.2283325, tolerance: 626810510.2533692
  positive)


...

Lasso(alpha=62, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=24, selection='cyclic',
      tol=0.0001, warm_start=False)

Alpha: 62
Lasso Test Score: 0.9158952230116779
Lasso MSE: 22558.904496134655


Lasso(alpha=63, copy_X=True, fit_intercept=True, max_iter=1000, normalize=False,
      positive=False, precompute=False, random_state=24, selection='cyclic',
      tol=0.0001, warm_start=False)

Alpha: 63
Lasso Test Score: 0.9158424816959244
Lasso MSE: 22565.976626162163


{33: [22,
  28,
  30,
  31,
  33,
  34,
  37,
  39,
  40,
  43,
  44,
  47,
  49,
  51,
  52,
  54,
  61,
  63,
  65,
  66,
  68,
  74,
  82,
  91,
  92,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  104,
  111,
  112,
  113,
  114,
  116,
  117,
  119,
  120,
  121,
  122,
  123,
  124,
  125,
  127,
  128,
  130,
  131,
  133,
  136,
  138,
  139,
  140,
  141,
  142,
  144,
  146,
  147,
  148,
  149,
  152,
  155,
  156,
  161,
  167,
  173,
  177,
  178,
  179,
  180,
  183,
  184,
  186,
  187,
  188,
  189,
  190,
  196,
  200,
  202,
  204,
  205,
  207,
  209,
  210,
  211,
  212,
  213,
  214,
  217,
  218,
  222,
  226,
  232,
  239,
  241,
  242,
  245,
  247,
  248,
  249,
  254,
  257,
  263,
  265,
  271,
  273,
  274,
  278,
  280,
  281,
  282],
 34: [22,
  28,
  30,
  31,
  33,
  34,
  37,
  39,
  40,
  43,
  44,
  47,
  49,
  51,
  52,
  54,
  61,
  63,
  65,
  66,
  68,
  74,
  82,
  91,
  92,
  95,
  96,
  97,
  98,
  99,
  100,
  101,
  102,
  103,
  104,
 

create dict with key alpha and value of column names to drop

In [14]:
features_to_drop_names = {}

for i in features_to_drop:
    feature_names = []
    for j in features_to_drop[i]:
        feature_names.append(housing.columns[j])
    features_to_drop_names[i]=feature_names

features_to_drop_names

{33: ['GarageYrBlt',
  '3SsnPorch',
  'PoolArea',
  'MiscVal',
  'MSSubClass_40',
  'MSSubClass_45',
  'MSSubClass_70',
  'MSSubClass_80',
  'MSSubClass_85',
  'MSSubClass_160',
  'MSSubClass_180',
  'MSZoning_RH',
  'MSZoning_RM',
  'Alley_No_Alley',
  'Alley_Pave',
  'LotShape_IR3',
  'LotConfig_FR3',
  'LandSlope_Mod',
  'Neighborhood_Blueste',
  'Neighborhood_BrDale',
  'Neighborhood_ClearCr',
  'Neighborhood_MeadowV',
  'Neighborhood_SWISU',
  'Condition1_PosA',
  'Condition1_PosN',
  'Condition1_RRNe',
  'Condition1_RRNn',
  'Condition2_Feedr',
  'Condition2_Norm',
  'Condition2_PosA',
  'Condition2_PosN',
  'Condition2_RRAe',
  'Condition2_RRAn',
  'BldgType_2fmCon',
  'HouseStyle_2.5Unf',
  'HouseStyle_2Story',
  'HouseStyle_SFoyer',
  'HouseStyle_SLvl',
  'RoofStyle_Gambrel',
  'RoofStyle_Hip',
  'RoofStyle_Shed',
  'RoofMatl_CompShg',
  'RoofMatl_Membran',
  'RoofMatl_Metal',
  'RoofMatl_Roll',
  'RoofMatl_Tar&Grv',
  'RoofMatl_WdShake',
  'Exterior1st_AsphShn',
  'Exterior1s

In [15]:
drop_difference_dict = {}
for i in range(34,64):
    new_dropped = []
    for name in features_to_drop_names[i]:
        if name not in features_to_drop_names[i-1]:
            print(name, i)
            new_dropped.append(name)
    drop_difference_dict[i] = new_dropped

            
            
        

Condition2_RRNn 34
Foundation_Wood 34
2ndFlrSF 35
Fence_No_Fence 36
MiscFeature_Shed 36
MoSold_11 36
BsmtFinType1_Unf 37
Condition1_Feedr 38
BsmtExposure_Mn 38
HouseStyle_1.5Unf 39
SaleCondition_AdjLand 39
LandSlope_Sev 41
Neighborhood_NPkVill 41
GarageCond_Gd 41
LandContour_Low 42
LotShape_IR2 43
Heating_GasA 43
Foundation_Stone 44
EnclosedPorch 45
HouseStyle_2.5Fin 45
MoSold_9 45
MoSold_12 46
GarageCond_TA 46
Exterior2nd_Wd Sdng 50
Electrical_FuseF 50
Exterior2nd_Stone 52
Street_Pave 53
RoofStyle_Mansard 53
BsmtHalfBath 54
BsmtFinType2_LwQ 54
GarageQual_Fa 54
BsmtFinType1_LwQ 55
MSSubClass_30 56
GarageType_Basment 56
SaleType_CWD 57
HeatingQC_Fa 58
MSSubClass_190 60
YrSold_2009 60
Neighborhood_IDOTRR 61
FireplaceQu_No_FP 61
Neighborhood_Sawyer 63


some increases in alpha actually lowers the number of features dropped. good to note

In [16]:
housing.shape
housing.sample(2)

(1444, 284)

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_No_Alley,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CemntBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_No_Bsmt,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_No_Bsmt,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_No_G,PavedDrive_P,PavedDrive_Y,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_No_Fence,MiscFeature_No_MF,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_No_Bsmt,BsmtQual_TA,BsmtCond_Gd,BsmtCond_No_Bsmt,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_No_Bsmt,HeatingQC_Fa,HeatingQ

In [17]:
dropped_36 = housing.drop(features_to_drop_names[36], axis=1)

In [18]:
dropped_50 = housing.drop(features_to_drop_names[50], axis=1)
dropped_63 = housing.drop(features_to_drop_names[63], axis=1)

In [19]:
dropped_36.shape[1]

163

In [20]:
random_state=24
k6 = KFold(n_splits=6, shuffle=True, random_state=random_state)

train = []
test = []

for train_idx, val_idx in k6.split(housing):
    train.append(train_idx)
    test.append(val_idx)

ridge = Ridge(alpha=2, random_state=random_state, max_iter=10000)

score_log = []

for df in (dropped_36,dropped_50,dropped_63):
    for i in range(6):
        single_log = []
        
        X_train = df.iloc[train[i],:-1]
        X_test = df.iloc[test[i],:-1]
        y_train = df.iloc[train[i],-1]
        y_test = df.iloc[test[i],-1]
        
        
        ridge.fit(X_train,y_train) 
        single_log.append(i)
        single_log.append(df.shape[1])
        single_log.append(ridge.score(X_train,y_train))
        single_log.append(ridge.score(X_test, y_test))
        single_log.append(mean_squared_error(y_test, ridge.predict(X_test))**0.5)
        
        score_log.append(single_log)

score_df = pd.DataFrame(score_log, columns=['Fold','Columns','TrainScore','TestScore','MSE'])


    
        
    
    
    

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

...

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

In [21]:
score_df.groupby('Columns').mean()[['MSE','TestScore']]

,MSE,TestScore
Columns,,
132,22577.210,0.910
146,22639.607,0.910
163,22705.818,0.909


lets test not scaling/standardizing

In [22]:
full_housing = pd.read_csv('full_ohe_no_outlier.csv', index_col=0)
full_housing.shape
full_housing.sample(5)

(1444, 284)

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_No_Alley,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CemntBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_No_Bsmt,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_No_Bsmt,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_No_G,PavedDrive_P,PavedDrive_Y,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_No_Fence,MiscFeature_No_MF,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_No_Bsmt,BsmtQual_TA,BsmtCond_Gd,BsmtCond_No_Bsmt,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_No_Bsmt,HeatingQC_Fa,HeatingQ

In [34]:
random_state=24
k6 = KFold(n_splits=6, shuffle=True, random_state=random_state)

train = []
test = []

for train_idx, val_idx in k6.split(full_housing):
    train.append(train_idx)
    test.append(val_idx)

ridge = Ridge(alpha=2, random_state=random_state, max_iter=10000)
lasso = Lasso(alpha=60, random_state=random_state, max_iter=10000)

score_log = []

for model in (ridge,lasso):
    for i in range(6):
        single_log = []

        X_train = full_housing.iloc[train[i],:-1]
        X_test = full_housing.iloc[test[i],:-1]
        y_train = full_housing.iloc[train[i],-1]
        y_test = full_housing.iloc[test[i],-1]


        model.fit(X_train,y_train) 
        single_log.append(i)
        single_log.append(str(type(model))[-7:])
        single_log.append(model.score(X_train,y_train))
        single_log.append(model.score(X_test, y_test))
        single_log.append(mean_squared_error(y_test, model.predict(X_test))**0.5)

        score_log.append(single_log)

score_df = pd.DataFrame(score_log, columns=['Fold','Regressor','TrainScore','TestScore','MSE'])


    
        
    
    
    

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

...

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

In [35]:
score_df

,Fold,Regressor,TrainScore,TestScore,MSE
0,0,Ridge'>,0.941,0.917,23246.435
1,1,Ridge'>,0.941,0.913,18520.268
2,2,Ridge'>,0.940,0.927,20246.900
3,3,Ridge'>,0.948,0.869,24507.710
4,4,Ridge'>,0.941,0.913,25355.905
5,5,Ridge'>,0.943,0.902,25791.127
6,0,Lasso'>,0.935,0.915,23456.417
7,1,Lasso'>,0.936,0.907,19100.405
8,2,Lasso'>,0.935,0.928,20173.530
9,3,Lasso'>,0.942,0.876,23899.484


In [36]:
score_df.groupby('Regressor').mean()

,Fold,TrainScore,TestScore,MSE
Regressor,,,,
Lasso'>,2.500,0.936,0.908,22815.544
Ridge'>,2.500,0.942,0.907,22944.724


lets test vs log(y)

In [50]:
random_state=24
k6 = KFold(n_splits=6, shuffle=True, random_state=random_state)

train = []
test = []

for train_idx, val_idx in k6.split(full_housing):
    train.append(train_idx)
    test.append(val_idx)

ridge = Ridge(alpha=2, random_state=random_state, max_iter=10000)
lasso = Lasso(alpha=60, random_state=random_state, max_iter=10000)

score_log = []

for model in (ridge,lasso):
    for i in range(6):
        single_log = []

        X_train = full_housing.iloc[train[i],:-1]
        X_test = full_housing.iloc[test[i],:-1]
        y_train = np.log1p(full_housing.iloc[train[i],-1])
        y_test = np.log1p(full_housing.iloc[test[i],-1])


        model.fit(X_train,y_train) 
        single_log.append(i)
        single_log.append(str(type(model))[-7:])
        single_log.append(model.score(X_train,y_train))
        single_log.append(model.score(X_test, y_test))
        y_pred = []
        for pred in model.predict(X_test):
            y_pred.append(exp(pred))
        single_log.append(mean_squared_error(full_housing.iloc[test[i],-1], y_pred)**0.5)

        score_log.append(single_log)

score_df = pd.DataFrame(score_log, columns=['Fold','Regressor','TrainScore','TestScore','MSE'])


    
        
    
    
    

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

...

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

In [51]:
score_df

,Fold,Regressor,TrainScore,TestScore,MSE
0,0,Ridge'>,0.946,0.923,21780.468
1,1,Ridge'>,0.950,0.899,16121.995
2,2,Ridge'>,0.947,0.924,18125.646
3,3,Ridge'>,0.953,0.875,21346.123
4,4,Ridge'>,0.946,0.927,19464.060
5,5,Ridge'>,0.946,0.923,23172.927
6,0,Lasso'>,0.407,0.507,59991.405
7,1,Lasso'>,0.430,0.404,48128.644
8,2,Lasso'>,0.425,0.399,62359.677
9,3,Lasso'>,0.439,0.388,54402.707


In [40]:
housing.sample(2)

,LotFrontage,LotArea,OverallQual,OverallCond,YearBuilt,YearRemodAdd,MasVnrArea,BsmtFinSF1,BsmtFinSF2,BsmtUnfSF,1stFlrSF,2ndFlrSF,LowQualFinSF,GrLivArea,BsmtFullBath,BsmtHalfBath,FullBath,HalfBath,BedroomAbvGr,KitchenAbvGr,TotRmsAbvGrd,Fireplaces,GarageYrBlt,GarageCars,GarageArea,WoodDeckSF,OpenPorchSF,EnclosedPorch,3SsnPorch,ScreenPorch,PoolArea,MiscVal,MSSubClass_30,MSSubClass_40,MSSubClass_45,MSSubClass_50,MSSubClass_60,MSSubClass_70,MSSubClass_75,MSSubClass_80,MSSubClass_85,MSSubClass_90,MSSubClass_120,MSSubClass_160,MSSubClass_180,MSSubClass_190,MSZoning_FV,MSZoning_RH,MSZoning_RL,MSZoning_RM,Street_Pave,Alley_No_Alley,Alley_Pave,LotShape_IR2,LotShape_IR3,LotShape_Reg,LandContour_HLS,LandContour_Low,LandContour_Lvl,LotConfig_CulDSac,LotConfig_FR2,LotConfig_FR3,LotConfig_Inside,LandSlope_Mod,LandSlope_Sev,Neighborhood_Blueste,Neighborhood_BrDale,Neighborhood_BrkSide,Neighborhood_ClearCr,Neighborhood_CollgCr,Neighborhood_Crawfor,Neighborhood_Edwards,Neighborhood_Gilbert,Neighborhood_IDOTRR,Neighborhood_MeadowV,Neighborhood_Mitchel,Neighborhood_NAmes,Neighborhood_NPkVill,Neighborhood_NWAmes,Neighborhood_NoRidge,Neighborhood_NridgHt,Neighborhood_OldTown,Neighborhood_SWISU,Neighborhood_Sawyer,Neighborhood_SawyerW,Neighborhood_Somerst,Neighborhood_StoneBr,Neighborhood_Timber,Neighborhood_Veenker,Condition1_Feedr,Condition1_Norm,Condition1_PosA,Condition1_PosN,Condition1_RRAe,Condition1_RRAn,Condition1_RRNe,Condition1_RRNn,Condition2_Feedr,Condition2_Norm,Condition2_PosA,Condition2_PosN,Condition2_RRAe,Condition2_RRAn,Condition2_RRNn,BldgType_2fmCon,BldgType_Duplex,BldgType_Twnhs,BldgType_TwnhsE,HouseStyle_1.5Unf,HouseStyle_1Story,HouseStyle_2.5Fin,HouseStyle_2.5Unf,HouseStyle_2Story,HouseStyle_SFoyer,HouseStyle_SLvl,RoofStyle_Gable,RoofStyle_Gambrel,RoofStyle_Hip,RoofStyle_Mansard,RoofStyle_Shed,RoofMatl_CompShg,RoofMatl_Membran,RoofMatl_Metal,RoofMatl_Roll,RoofMatl_Tar&Grv,RoofMatl_WdShake,RoofMatl_WdShngl,Exterior1st_AsphShn,Exterior1st_BrkComm,Exterior1st_BrkFace,Exterior1st_CBlock,Exterior1st_CemntBd,Exterior1st_HdBoard,Exterior1st_ImStucc,Exterior1st_MetalSd,Exterior1st_Plywood,Exterior1st_Stone,Exterior1st_Stucco,Exterior1st_VinylSd,Exterior1st_Wd Sdng,Exterior1st_WdShing,Exterior2nd_AsphShn,Exterior2nd_Brk Cmn,Exterior2nd_BrkFace,Exterior2nd_CBlock,Exterior2nd_CemntBd,Exterior2nd_HdBoard,Exterior2nd_ImStucc,Exterior2nd_MetalSd,Exterior2nd_Other,Exterior2nd_Plywood,Exterior2nd_Stone,Exterior2nd_Stucco,Exterior2nd_VinylSd,Exterior2nd_Wd Sdng,Exterior2nd_Wd Shng,MasVnrType_BrkFace,MasVnrType_None,MasVnrType_Stone,Foundation_CBlock,Foundation_PConc,Foundation_Slab,Foundation_Stone,Foundation_Wood,BsmtFinType1_BLQ,BsmtFinType1_GLQ,BsmtFinType1_LwQ,BsmtFinType1_No_Bsmt,BsmtFinType1_Rec,BsmtFinType1_Unf,BsmtFinType2_BLQ,BsmtFinType2_GLQ,BsmtFinType2_LwQ,BsmtFinType2_No_Bsmt,BsmtFinType2_Rec,BsmtFinType2_Unf,Heating_GasA,Heating_GasW,Heating_Grav,Heating_OthW,Heating_Wall,CentralAir_Y,Electrical_FuseF,Electrical_FuseP,Electrical_Mix,Electrical_SBrkr,Functional_Maj2,Functional_Min1,Functional_Min2,Functional_Mod,Functional_Sev,Functional_Typ,GarageType_Attchd,GarageType_Basment,GarageType_BuiltIn,GarageType_CarPort,GarageType_Detchd,GarageType_No_G,PavedDrive_P,PavedDrive_Y,Fence_GdWo,Fence_MnPrv,Fence_MnWw,Fence_No_Fence,MiscFeature_No_MF,MiscFeature_Othr,MiscFeature_Shed,MiscFeature_TenC,SaleType_CWD,SaleType_Con,SaleType_ConLD,SaleType_ConLI,SaleType_ConLw,SaleType_New,SaleType_Oth,SaleType_WD,SaleCondition_AdjLand,SaleCondition_Alloca,SaleCondition_Family,SaleCondition_Normal,SaleCondition_Partial,MoSold_2,MoSold_3,MoSold_4,MoSold_5,MoSold_6,MoSold_7,MoSold_8,MoSold_9,MoSold_10,MoSold_11,MoSold_12,YrSold_2007,YrSold_2008,YrSold_2009,YrSold_2010,ExterQual_Fa,ExterQual_Gd,ExterQual_TA,ExterCond_Fa,ExterCond_Gd,ExterCond_Po,ExterCond_TA,BsmtQual_Fa,BsmtQual_Gd,BsmtQual_No_Bsmt,BsmtQual_TA,BsmtCond_Gd,BsmtCond_No_Bsmt,BsmtCond_Po,BsmtCond_TA,BsmtExposure_Gd,BsmtExposure_Mn,BsmtExposure_No,BsmtExposure_No_Bsmt,HeatingQC_Fa,HeatingQ

that worked out terribly. lets try vs a scaled df

In [47]:
random_state=24
k6 = KFold(n_splits=6, shuffle=True, random_state=random_state)

train = []
test = []

for train_idx, val_idx in k6.split(housing):
    train.append(train_idx)
    test.append(val_idx)

ridge = Ridge(alpha=2, random_state=random_state, max_iter=10000)
lasso = Lasso(alpha=60, random_state=random_state, max_iter=10000)

score_log = []

for model in (ridge,lasso):
    for i in range(6):
        single_log = []

        X_train = housing.iloc[train[i],:-1]
        X_test = housing.iloc[test[i],:-1]
        y_train = np.log1p(housing.iloc[train[i],-1])
        y_test = np.log1p(housing.iloc[test[i],-1])


        model.fit(X_train,y_train) 
        single_log.append(i)
        single_log.append(str(type(model))[-7:])
        single_log.append(model.score(X_train,y_train))
        single_log.append(model.score(X_test, y_test))
        y_pred = []
        for pred in model.predict(X_test):
            y_pred.append(exp(pred))
        single_log.append(mean_squared_error(housing.iloc[test[i],-1], y_pred)**0.5)

        score_log.append(single_log)

score_df = pd.DataFrame(score_log, columns=['Fold','Regressor','TrainScore','TestScore','MSE'])


    
        
    
    
    

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

Ridge(alpha=2, copy_X=True, fit_intercept=True, max_iter=10000, normalize=False,
      random_state=24, solver='auto', tol=0.001)

...

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

Lasso(alpha=60, copy_X=True, fit_intercept=True, max_iter=10000,
      normalize=False, positive=False, precompute=False, random_state=24,
      selection='cyclic', tol=0.0001, warm_start=False)

In [48]:
score_df

,Fold,Regressor,TrainScore,TestScore,MSE
0,0,Ridge'>,0.945,0.923,21124.982
1,1,Ridge'>,0.949,0.895,16637.833
2,2,Ridge'>,0.945,0.919,18943.017
3,3,Ridge'>,0.952,0.876,21173.285
4,4,Ridge'>,0.944,0.926,20333.983
5,5,Ridge'>,0.944,0.931,20734.441
6,0,Lasso'>,0.000,-0.019,83875.923
7,1,Lasso'>,0.000,-0.026,62766.766
8,2,Lasso'>,0.000,-0.002,76703.955
9,3,Lasso'>,0.000,-0.003,68286.197


In [49]:
housing.shape

(1444, 284)